### Info
Author: S. Bakker

Date: January 2023

#### Importing packages and setting project data based on DEM

In [3]:
import os
import gzip
import shutil
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.fill import fillnodata
import numpy as np
# from rasterio.warp import reproject, Resampling, calculate_default_transform

WD_INPUT = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Inputmaps_5m/'
sim_domain = 'SimDomain_5m.asc'

NO_DATA = -9999


PROJCS["ETRS89_ETRS_LAEA",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",52],PARAMETER["longitude_of_center",10],PARAMETER["false_easting",4321000],PARAMETER["false_northing",3210000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [2]:
WD = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Netherlands/'
WD_tiles = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Netherlands/Loose_tiles/'

small_tiles = os.listdir(WD_tiles)
print(small_tiles)

['M_68DZ1.TIF', 'M_69AN2.TIF', 'M_69BN1.TIF', 'M_69BN2.TIF', 'M_69BZ1.TIF', 'M_69BZ2.TIF', 'M_69DN2.TIF', 'M_69EN1.TIF', 'M_69EZ1.TIF', 'M_69EZ2.TIF', 'M_69GN1.TIF', 'M_69GN2.TIF', 'NoDataM_68DZ1.TIF', 'NoDataM_69AN2.TIF', 'NoDataM_69BN1.TIF', 'NoDataM_69BN2.TIF', 'NoDataM_69BZ1.TIF', 'NoDataM_69BZ2.TIF', 'NoDataM_69DN2.TIF', 'NoDataM_69EN1.TIF', 'NoDataM_69EZ1.TIF', 'NoDataM_69EZ2.TIF', 'NoDataM_69GN1.TIF', 'NoDataM_69GN2.TIF']


### Making nodata -9999 so that the file can be float32 isntead of float64 
This makes running the next steps a lot quicker (otherwise they would be possible due to size of the files) 

In [14]:
for i in range(1, (len(small_tiles))):
    with rasterio.open(f'{WD_tiles}{small_tiles[i]}', 'r') as ras:
        array = ras.read(1)
        array = np.where(array == ras.nodata, NO_DATA, array)
    with rasterio.open(f'{WD_tiles}NoData{small_tiles[i]}', 
                   'w', 
                   driver = 'GTiff',
                   dtype = 'float32',
                   height = ras.height,
                   width = ras.width,
                   count= 1,
                   crs = ras.crs,
                   transform = ras.transform,
                  nodata = NO_DATA) as dst:
        dst.write(array, indexes = 1)

In [3]:
# with rasterio.open(f'{WD_tiles}{small_tiles[0]}', 'r') as ras:
#     array = ras.read(1)
#     array = np.where(array == ras.nodata, NO_DATA, array)
#     with rasterio.open(f'{WD}NL_Mergedfile.tif', 
#                        'w', 
#                        driver = 'GTiff',
#                        dtype = 'float32',
#                        height = ras.height,
#                        width = ras.width,
#                        count= 1,
#                        crs = ras.crs,
#                        transform = ras.transform,
#                       nodata = NO_DATA) as dst:
#         dst.write(array, indexes = 1)
        
for i in range(1, (len(small_tiles))):
    with rasterio.open(f'{WD}NL_Mergedfile.tif', 'r', dtype = 'float32') as ras_1:
        with rasterio.open(f'{WD_tiles}NoData{small_tiles[i]}', 'r', dtype = 'float32') as ras_2:
            merged, trans = merge([ras_1, ras_2], nodata = ras_2.nodata, dtype = 'float32')
#             merged = np.where(merged == ras_2.nodata, NO_DATA, merged)
            
    with rasterio.open(f'{WD}NL_Mergedfile.tif', 
                       'w', 
                       driver = 'GTiff',
                       dtype = 'float32',
                       height = merged.shape[1],
                       width = merged.shape[2],
                       count= 1,
                       crs = ras_1.crs,
                       transform = trans,
                       nodata = NO_DATA
                      ) as dst:
        dst.write(merged)
        

dst.close

RasterioIOError: C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Netherlands/Loose_tiles/NoDataNoDataM_68DZ1.TIF: No such file or directory